<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Chest%20X_Ray/Chest_X_Ray_by_MobileNetV2_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ============================================================
# FAST CHEST X-RAY CLASSIFICATION (NORMAL vs PNEUMONIA)
# MobileNetV2 | Simplified | Fast Convergence
# ============================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

# ----------------------------
# 1. Mount Google Drive
# ----------------------------
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------
# 1. Configuration
# ----------------------------
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5
LR = 1e-4
DATA_DIR = "/content/drive/MyDrive/Chest X_Ray"  # contains NORMAL / PNEUMONIA
SEED = 42

# ----------------------------
# 2. Data Generators
# ----------------------------
# Minimal augmentation → faster + more stable convergence
train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = train_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=SEED
)

val_data = train_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# ----------------------------
# 3. Build MobileNetV2 Model
# ----------------------------
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(*IMG_SIZE, 3)
)

# Freeze backbone (very important for fast convergence)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=LR),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# model.summary()

# ----------------------------
# 4. Callbacks (Stability + Speed)
# ----------------------------
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(patience=2, factor=0.3, verbose=1)
]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 4686 images belonging to 2 classes.
Found 1170 images belonging to 2 classes.


In [6]:
from sklearn.metrics import (
    confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score,
    roc_curve, auc
)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ----------------------------
# Helper Function: Evaluate Generator
# ----------------------------
def evaluate_generator(model, generator):
    generator.reset()
    y_true = generator.classes
    y_prob = model.predict(generator, verbose=0).ravel()
    y_pred = (y_prob > 0.5).astype(int)

    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred)
    }

    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)

    return metrics, cm, fpr, tpr, roc_auc

# ----------------------------
# 1. Evaluate Train & Test
# ----------------------------
train_metrics, cm_train, fpr_train, tpr_train, auc_train = evaluate_generator(model, train_data)
test_metrics,  cm_test,  fpr_test,  tpr_test,  auc_test  = evaluate_generator(model, val_data)

KeyboardInterrupt: 

In [ ]:
# ----------------------------
# 2. Metrics Table
# ----------------------------
df_results = pd.DataFrame([
    ["Train", *train_metrics.values()],
    ["Test",  *test_metrics.values()]
], columns=["Dataset", "Accuracy", "Precision", "Recall", "F1"])

print("\nPerformance Summary:")
display(df_results)

# ----------------------------
# 3. Confusion Matrices (Subplots)
# ----------------------------
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.imshow(cm_train, cmap="Blues")
plt.title("Train Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(cm_test, cmap="Blues")
plt.title("Test Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()

plt.tight_layout()
plt.show()

# ----------------------------
# ROC Curve (Train & Test on One Plot)
# ----------------------------
plt.figure(figsize=(6,6))

plt.plot(
    fpr_train, tpr_train,
    label=f"Train ROC (AUC = {auc_train:.3f})",
    linewidth=2
)

plt.plot(
    fpr_test, tpr_test,
    label=f"Test ROC (AUC = {auc_test:.3f})",
    linewidth=2
)

plt.plot([0, 1], [0, 1], 'k--', linewidth=1)

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Train vs Test")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()
